In [8]:
import pandas as pd
import numpy as np

# Data loading and setting up
df = pd.read_csv('d.csv')
availabilities = df[df.columns[2]].astype(str)
months = df[df.columns[0]].ffill().astype(str)
date = df[df.columns[1]].astype(np.uint8)
df = df.drop(df.columns[range(3)], axis=1)
data = df.to_numpy().astype(np.uint8)

# SLink mat setup
mat = np.zeros((19,19)).astype(np.uint8)
np.fill_diagonal(mat, 10)
mat[11, 11] = 6
mat[9, 9] = 4

weights = np.ones((19,))

listrange = lambda x,y: list(range(x, y))
day_chars = [*listrange(0, 2), *listrange(3,12), *listrange(14, 17), 18]
night_chars = sorted(list(set(listrange(0,19))-set(day_chars)))


## Weights for Kou

In [5]:
# Kou  setup

weights[1] = 1.6
weights[3] = 1.7
weights[5] = 1.1
weights[6] = 1.1
weights[8] = 1.2
weights[14] = 1.1
weights[15] = 1.5
weights[18] = 1.2
df.columns[np.argsort(weights)[::-1]]

Index(['Yukiko Amagi', 'Chie Satonaka', 'Naoki Konishi', 'Naoto Shirogane',
       'Kanji Tatsumi', 'Ai Ebihara', 'Kou Ichijo', 'Ayane Matsunaga',
       'Eri Minami', 'Nanako Dojima', 'Fox', 'Marie', 'Shu Nakajima',
       'Rise Kujikawa', 'Tohru Adachi', 'Sayoko Uehara', 'Ryotaro Dojima',
       'Hisano Kuroda', 'Yosuke Hanamura'],
      dtype='object')

## Weights for Daisuke

In [9]:
# Daisuke setup
df.columns[np.argsort(weights)[::-1]]

Index(['Naoto Shirogane', 'Kanji Tatsumi', 'Chie Satonaka', 'Nanako Dojima',
       'Yukiko Amagi', 'Fox', 'Daisuke Nagase', 'Ayane Matsunaga',
       'Eri Minami', 'Marie', 'Shu Nakajima', 'Rise Kujikawa', 'Tohru Adachi',
       'Sayoko Uehara', 'Ryotaro Dojima', 'Ai Ebihara', 'Naoki Konishi',
       'Hisano Kuroda', 'Yosuke Hanamura'],
      dtype='object')

## Simulation

In [10]:
def iterate(r, data, mat):
    s = data.sum(axis=0)
    r = np.divide(r, s, out=np.zeros_like(r).astype(np.float32), where=s!=0)
    res = mat@r
    maxdata = np.argmax(res)
    if res[maxdata] <= 0: return mat, None
    curr = mat[maxdata, maxdata]
    if curr <= 0: return mat, None
    mat[maxdata, maxdata] = curr - 1
    return mat, maxdata

selecteds = []
for i, r in enumerate(data.copy()):
    r = r * weights
    if i == 105:
        mat[9, 9] = 6 + mat[9, 9]
    if availabilities[i] == 'b':
        a = r.copy()
        a[night_chars] = 0
        d = data[i:].copy()
        d[:, night_chars] = 0
        mat, selected1 = iterate(a, d, mat)
        a = r.copy()
        a[day_chars] = 0
        d = data[i:].copy()
        d[:, day_chars] = 0
        mat, selected2 = iterate(a, d, mat)
        selected = (selected1, selected2)
    else:
        mat, selected = iterate(r, data[i:], mat)
        selected = (selected, None)
    selecteds.append(selected)
print(np.diag(mat))
print(df.columns[np.diag(mat) > 0], np.diag(mat)[np.diag(mat) > 0])
print(np.where(np.diag(mat) > 0))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Index([], dtype='object') []
(array([], dtype=int64),)


In [11]:
names = df.columns.to_numpy().copy()
none = names.shape[0]
names = np.append(names, 'None')
p = []
for i, (d, m) in enumerate(zip(date, months)):
    s1, s2 = selecteds[i]
    if s1 is None: s1 = none
    if s2 is None:
        p.append(names[s1])
    else:
        p.append(f'Day: {names[s1]}, Night: {names[s2]}')

import csv
with open('od.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(zip(months,date,p))